In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Sun Jun  5 16:35:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   44C    P8    38W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_us = db.rm_patent_us
    db_rm_cn = db.rm_patent_cn
    db_rm_de = db.rm_patent_de
    
    db_rm_us_2015 = db.rm_patent_us_2015
    db_rm_cn_2015 = db.rm_patent_cn_2015
    db_rm_de_2015 = db.rm_patent_de_2015

except errors.ConnectionFailure as err:
    print(err)

In [5]:
db_rm_us_2015

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'rm_patent_us_2015')

# DE-CN-US

## 2015

### DE

In [6]:
data_assignee = db_rm_de_2015.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [7]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_de_2015.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Samsung Electronics Co Ltd,H04W4,6
1,Samsung Electronics Co Ltd,H04W12,5
2,Samsung Electronics Co Ltd,20,3
3,Samsung Electronics Co Ltd,H04W52,2
4,Samsung Electronics Co Ltd,H04M15,2


In [8]:
df_cpc.shape

(243, 3)

In [9]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [10]:
# 轉成company-cpc 的總數表格
table_de = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_de = table_de.droplevel(level=0, axis=1)
table_de = table_de.rename(columns={'':'company'})

In [11]:
# 取有英文的部份
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

cpc_n,company,A42,A61,B27,G01,G02,G05,G06,G07,G08,G09,H01,H02,H03,H04,Y02
0,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
1,Audi AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Deutsche Telekom AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0
3,EMDEDESIGN GmbH,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,IFM DATALINK GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
final_table_de.shape

(21, 16)

In [13]:
final_table_de.to_csv('rm_company_cpc_de_2015.csv',index=0)

把final_table資料存進MongoDB

In [14]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_de_2015

In [15]:
# title
conn_db_collection_rm.insert_one({"company":final_table_de.columns.tolist()[0],"cpc":final_table_de.columns.tolist()[1:]})

# 內容
for k in final_table_de.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

In [36]:
# title
conn_db_collection_rm.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### CN

In [17]:
data_assignee = db_rm_cn_2015.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [18]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_cn_2015.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Chengdu Qinchuan IoT Technology Co Ltd,H04L67,21
1,Chengdu Qinchuan IoT Technology Co Ltd,12,13
2,Chengdu Qinchuan IoT Technology Co Ltd,G07F15,8
3,Chengdu Qinchuan IoT Technology Co Ltd,06,7
4,Chengdu Qinchuan IoT Technology Co Ltd,08,6


In [19]:
df_cpc.shape

(3552, 3)

In [20]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [21]:
# 轉成company-cpc 的總數表格
table_cn = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_cn = table_cn.droplevel(level=0, axis=1)
table_cn = table_cn.rename(columns={'':'company'})

In [22]:
# 取有英文的部份
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

cpc_n,company,A01,A23,A41,A42,A43,A44,A45,A47,A61,...,G10,G11,G16,H01,H02,H03,H04,H05,Y02,Y04
0,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3b Technology Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,15.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3d-Oxset Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ANHUI BAIBAITONG INTERNET OF THINGS TECHNOLOGY...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,"ANHUI SHARETRONIC IOT TECHNOLOGY CO., LTD.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
final_table_cn.shape

(311, 79)

In [24]:
final_table_cn.to_csv('rm_company_cpc_cn_2015.csv',index=0)

把final_table資料存進MongoDB

In [25]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_cn_2015

In [26]:
# title
conn_db_collection_rm.insert_one({"company":final_table_cn.columns.tolist()[0],"cpc":final_table_cn.columns.tolist()[1:]})

# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### US

In [27]:
data_assignee = db_rm_us_2015.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [28]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_us_2015.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Samsung Electronics Co Ltd,H04W4,233
1,Samsung Electronics Co Ltd,G06F3,231
2,Samsung Electronics Co Ltd,H01L27,112
3,Samsung Electronics Co Ltd,02,95
4,Samsung Electronics Co Ltd,H04L67,88


In [29]:
df_cpc.shape

(15119, 3)

In [30]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [31]:
# 轉成company-cpc 的總數表格
table_us = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_us = table_us.droplevel(level=0, axis=1)
table_us = table_us.rename(columns={'':'company'})

In [32]:
# 取有英文的部份
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

cpc_n,company,A01,A23,A41,A42,A44,A46,A47,A61,A62,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3d-Oxides,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ABB Schweiz AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,1.0,0.0
2,ALLIACENSE Ltd LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,APPSFREEDOM Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,ARM Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,13.0,2.0,33.0,0.0,1.0,0.0,0.0


In [33]:
final_table_us.shape

(472, 58)

In [34]:
final_table_us.to_csv('rm_company_cpc_us_2015.csv',index=0)

把final_table資料存進MongoDB

In [35]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_us_2015

In [36]:
# title
conn_db_collection_rm.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### 合併

In [37]:
# concat de cn us
df_c = pd.concat([final_table_de,final_table_cn], join='outer')
df_c2 = pd.concat([df_c,final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2.head()

cpc_n,company,A01,A23,A41,A42,A43,A44,A45,A46,A47,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
1,Audi AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Deutsche Telekom AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
3,EMDEDESIGN GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,IFM DATALINK GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [38]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()
df_c2.head()

cpc_n,company,A01,A23,A41,A42,A43,A44,A45,A46,A47,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3b Technology Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3d-Oxides,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3d-Oxset Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ABB Schweiz AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,1.0,0.0


In [39]:
df_c2.to_csv('rm_company_cpc_all_2015.csv',index=0)

In [40]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_cpc_all_2015

In [41]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"cpc":df_c2.columns.tolist()[1:]})

# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})